# Area burned by forest fires since 1990
*June 8, 2022*

A look at what area is burned by forest fires every year, broken down by cause of fire.

First, import required modules and packages.

In [4]:
import requests
import geopandas
import pandas as pd
import json
import datawrappergraphics  # For showing datawrapper graphic at the end.
import logging

In [5]:
# Make HTTP request for the data and put into a geopandas dataframe.
raw = geopandas.read_file("https://cwfis.cfs.nrcan.gc.ca/downloads/nfdb/fire_pnt/current_version/NFDB_point.zip", driver="shapefile")

raw["type"] = "point"
raw

,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,ATTK_DATE,...,CFS_NOTE1,CFS_NOTE2,ACQ_DATE,SRC_AGY2,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM,geometry,type
0,BC,1953-G00041,None,59.96300,-128.17200,1953,5,26,1953-05-26,None,...,None,None,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1720729.937 1659437.590 0.000),point
1,BC,1950-R00028,None,59.31800,-132.17200,1950,6,22,1950-06-22,None,...,None,None,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1944086.504 1715216.943 0.000),point
2,BC,1950-G00026,None,59.87600,-131.92200,1950,6,4,1950-06-04,None,...,None,None,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1899364.952 1758151.328 0.000),point
3,BC,1951-R00097,None,59.76000,-132.80800,1951,7,15,1951-07-15,None,...,None,None,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1946555.868 1774479.072 0.000),point
4,BC,1952-G00116,None,59.43400,-126.17200,1952,6,12,1952-06-12,None,...,None,None,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1652703.809 1556259.590 0.000),point
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417128,NB,VI-4-22-37,Remembrance Day,46.90015,-67.42012,2020,11,10,2020-11-10,2020-11-10,...,None,None,2021-09-16,NB,7,7,Atlantic Maritime,Maritime de l'Atlantique,POINT Z (2053936.130 217935.898 0.000),point
417129,NB,WE-2-17-120,Scoudouc line,46.13146,-64.55874,2020,11,10,2020-11-10,2020-11-10,...,None,None,2021-09-16,NB,7,7,Atlantic Maritime,Maritime de l'Atlantique,POINT Z (2291118.976 238155.889 0.000),point
417130,NB,WE-2-17-121,Old Railway Bed,45.96538,-64.31922,2020,11,13,2020-11-13,2020-11-13,...,None,None,2021-09-16,NB,7,7,Atlantic Maritime,Maritime de l'Atlantique,POINT Z (2316402.004 230256.577 0.000),point
417131,NB,SU-3-21-220,Portobello Grass Fire,45.89920,-66.51822,2020,11,18,2020-11-18,2020-11-18,...,None,None,2021-09-16,NB,7,7,Atlantic Maritime,Maritime de l'Atlantique,POINT Z (2165330.333 146116.068 0.000),point


Now we pivot our raw data so it shows total area burned per year.

In [6]:
pivot = raw.pivot_table(index="YEAR", values="SIZE_HA", columns="CAUSE", aggfunc='sum')

pivot.tail(5)

CAUSE,H,H-PB,L,RE,U,n/a
YEAR,,,,,,
2016,119175.440301,6479.1538,7.078798e+05,NaN,486038.3871,0.5000
2017,236686.837611,2189.5560,3.324999e+06,NaN,117.2480,25430.6701
2018,288796.385354,37075.7970,1.986958e+06,NaN,16018.2750,2.0000
2019,40298.741339,308.1500,9.076780e+05,NaN,1325.1970,836604.3300
2020,135132.541158,1391.5200,7.838660e+04,NaN,3115.2317,206.2400


Now we group columns into human, lightning, and other causes.

In [7]:
pivot["human"] = round(pivot["H"] + pivot["H-PB"], 2)
pivot["lightning"] = round(pivot["L"], 2)
pivot["other"] = round(pivot["RE"] + pivot["U"] + pivot["n/a"], 2)

pivot = pivot[pivot.index >= 1990]

pivot[["human", "lightning", "other"]].tail(5)

CAUSE,human,lightning,other
YEAR,,,
2016,125654.59,707879.77,NaN
2017,238876.39,3324998.95,NaN
2018,325872.18,1986958.50,NaN
2019,40606.89,907678.01,NaN
2020,136524.06,78386.60,NaN


In [8]:
datawrappergraphics.Chart("7FCuD").show()

c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\IPython\core\display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
